# Example Notebook

This notebook provides an example environment where to analyze samples using the ``sample`` module.

In [ ]:
# Python imports.
import sys
import os

# Add path to 'signal-edges' package and perform imports.
sys.path.append(os.path.abspath(r"./../../src"))

from signal_edges.signal import VoltageSignal
from signal_edges.signal.sample import Sample, Waveform, SampleManager
from signal_edges.signal.generator import SignalGenerator
from signal_edges.signal.edges import IntPointPolicy
from signal_edges import plotter as sep

In [ ]:
# Initialize sample manager to local vault.
sample_manager = SampleManager(os.path.normpath("./samples"))

# Create generator with initial values.
sampling_frequency = 100000
generator = SignalGenerator(0, 1 / sampling_frequency, 100, 0, 90)

# Build signal with multiple pulses.
generator.add_flat(100)
generator.add_edge("falling", 0, 100)
generator.add_flat(100)
generator.add_edge("rising", 90, 100)
generator.repeat(1000000 // 400)

# Generate signal with some noise.
(signal_x, signal_y) = generator.generate()

# Store as sample one with the sample manager.
sample = sample_manager.new(0, [{"wid": 0, "hvalues": signal_x, "vvalues": signal_y, "sp": sampling_frequency}], overwrite=True)

# Create voltage signal from sample.
signal = VoltageSignal(sample.waveforms[0].hvalues, sample.waveforms[0].vvalues, "s", "V")

# Create filtered signal.
filtered_signal = signal.filters_elliptic(sample.waveforms[0].meta["sp"], 2, sample.waveforms[0].meta["sp"] // 2.5)

# Calculate state levels.
(state_levels, (hist_x, hist_y)) = filtered_signal.state_levels()

# Calculate edges.
edges = filtered_signal.edges(state_levels, IntPointPolicy.POLICY_0)

In [ ]:
# Set delimiters of the plot.
begin_plot_edge = edges[len(edges) // 2]
end_plot_edge = edges[(len(edges) // 2) + 2]

begin_plot = begin_plot_edge["hbegin"]
end_plot = end_plot_edge["hend"]
munits = 2

# Create subplot items for first row.
row_0_item_0 = ("signal", (begin_plot, end_plot, munits, "Original Signal #0", "gray", signal))
row_0_item_1 = ("signal", (begin_plot, end_plot, munits, "Filtered Signal #0", "red", filtered_signal))
row_0_item_2 = ("state_levels", (begin_plot, end_plot, munits, filtered_signal, state_levels))
row_0_item_3 = ("edges", (begin_plot, end_plot, munits, filtered_signal, edges))

# Build rows for plot.
row_0 = [row_0_item_0, row_0_item_1, row_0_item_2, row_0_item_3]

# Set cursors.
cursor_0 = sep.Cursor("A", begin_plot_edge["iintermediate"], 0, 0, ["Filtered Signal #0"], 6)
cursor_1 = sep.Cursor("B", end_plot_edge["iintermediate"], 0, 0, ["Filtered Signal #0"], 6)

# Build plot for all the signals with the state levels.
SampleManager.plot(
    row_0=row_0,
    points=["begin", "intermediate", "end"],
    levels=["low", "high", "intermediate"],
    cursors=[cursor_0, cursor_1]
)